# ETL

In [167]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 
import warnings
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from sqlalchemy import create_engine
warnings.filterwarnings('ignore')

In [168]:
df_hechos=pd.read_excel(r'Data/homicidios.xlsx')

In [169]:
df_hechos.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [170]:
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

In [171]:
print("Porcentaje de valores nulos:\n", df_hechos.isna().sum())
print("\nCantidad de filas duplicadas:\n", df_hechos.duplicated().sum())

Porcentaje de valores nulos:
 ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

Cantidad de filas duplicadas:
 0


In [172]:
df_hechos = df_hechos.map(lambda x: x.lower() if isinstance(x, str) else x)

In [173]:
faltantes = ['', ' ', '.', '...', 'sd', 'sd-sd', 'point (. .)']
df_hechos.replace(faltantes, np.nan, inplace=True)
df_hechos.infer_objects(copy=False)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4.0,av piedra buena y av fernandez de la cruz,avenida,...,NaN,"fernandez de la cruz, f., gral. av.","piedra buena av. y fernandez de la cruz, f., g...",8,point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,moto-auto,moto,auto
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1.0,av gral paz y av de los corrales,gral paz,...,NaN,de los corrales av.,"paz, gral. av. y de los corrales av.",9,point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,auto-pasajeros,auto,pasajeros
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7.0,av entre rios 2034,avenida,...,2034.0,NaN,entre rios av. 2034,1,point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,moto-auto,moto,auto
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0.0,av larrazabal y gral villegas conrado,avenida,...,NaN,"villegas, conrado, gral.","larrazabal av. y villegas, conrado, gral.",8,point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,moto-sd,moto,NaN
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5.0,av san juan y presidente luis saenz peña,avenida,...,NaN,"saenz pe?a, luis, pres.","san juan av. y saenz peã‘a, luis, pres.",1,point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,moto-pasajeros,moto,pasajeros
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17.0,av. riestra y mom,avenida,...,NaN,mom,riestra av. y mom,7,point (102728.60090138 98186.24929177),-58.43353773,-34.64561636,moto-auto,moto,auto
692,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1.0,au dellepiane y lacarra,autopista,...,NaN,lacarra av.,"dellepiane, luis, tte. gral. y lacarra av.",9,point (99624.29795829 97569.69801131),-58.46739825,-34.65117757,moto-auto,moto,auto
693,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0.0,av. gaona y terrada,avenida,...,NaN,terrada,gaona av. y terrada,11,point (99116.45492358 101045.23284826),-58.47293407,-34.61984745,moto-cargas,moto,cargas
694,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10.0,av. eva peron 4071,avenida,...,4071.0,NaN,"peron, eva av. 4071",9,point (99324.54463985 97676.26932409),-58.47066794,-34.65021673,auto-cargas,auto,cargas


In [174]:
df_hechos.isna().sum()

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       1
HH                         1
LUGAR_DEL_HECHO            1
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                 14
pos x                     12
pos y                     12
PARTICIPANTES              5
VICTIMA                    9
ACUSADO                   23
dtype: int64

In [175]:
df_hechos[df_hechos['pos x'].isna()][['LUGAR_DEL_HECHO', 'Calle','Dirección Normalizada']]

,LUGAR_DEL_HECHO,Calle,Dirección Normalizada
38,autopista lugones pk 10000,"lugones, leopoldo av.",NaN
106,au buenos aires - la plata km. 4,autopista buenos aires - la plata,NaN
119,NaN,NaN,NaN
139,autopista 25 de mayo,autopista 25 de mayo,autopista 25 de mayo
176,av. leopoldo lugones pkm 6900,"lugones, leopoldo av.","lugones, leopoldo av."
180,au perito moreno y ramal enlace au1/au6,autopista perito moreno,NaN
181,au dellepiane 2400,autopista dellepiane luis tte. gral.,NaN
256,au arturo frondizi pkm 3100,autopista 1 sur presidente arturo frondizi,autopista 1 sur presidente arturo frondizi
313,autopista lugones km 4.7,"lugones, leopoldo av.",NaN
546,"lugones, leopoldo av. km 6,1","lugones, leopoldo av.",NaN


In [176]:

geolocator = Nominatim(user_agent='mi_aplicacion')

def obtener_coordenadas(calle):
    try:
        location = geolocator.geocode(calle)
        if location:
            latitud = round(location.latitude, 6)
            longitud = round(location.longitude, 6)
            return latitud, longitud
        else:
            return None, None
    except GeocoderTimedOut:
        return obtener_coordenadas(calle)

# Crear un DataFrame con las calles y columnas adicionales para latitud y longitud
data = {
    'Calle': [
        'Avenida Lugones Leopoldo 10000, Buenos Aires, Argentina',      #38
        'Autopista Doctor Ricardo Balbín, Buenos Aires, Argentina',     #106
        'Autopista 25 de Mayo, Buenos Aires Argentina',                 #139
        'Avenida Lugones Leopoldo 6900, Buenos Aires, Argentina',       #176
        'Autopista Perito Moreno, Buenos Aires Argentina',              #180
        'Avenida Teniente General Dellepiane, Buenos Aires Argentina',  #181
        'Autopista Arturo Frondizi, Buenos Aires Argentina',            #256
        'Avenida Lugones Leopoldo, Buenos Aires, Argentina',            #313
        'Avenida Lugones Leopoldo 6100, Buenos Aires, Argentina',       #546
        'Murguiondo, Buenos Aires Argentina',                           #559
        'Autopista Doctor Ricardo Balbín, Buenos Aires, Argentina'      #621
    ],
    'Latitud': [None] * 11,
    'Longitud': [None] * 11
}

df = pd.DataFrame(data)

# Recorrer las calles del DataFrame y obtener las coordenadas
for index, row in df.iterrows():
    calle = row['Calle']
    latitud, longitud = obtener_coordenadas(calle)
    if latitud and longitud:
        df.at[index, 'Latitud'] = latitud
        df.at[index, 'Longitud'] = longitud
    else:
        print(f"No se pudo geocodificar la calle: {calle}")

# Mostrar el DataFrame resultante
df.head()

,Calle,Latitud,Longitud
0,"Avenida Lugones Leopoldo 10000, Buenos Aires, ...",-34.558958,-58.42098
1,"Autopista Doctor Ricardo Balbín, Buenos Aires,...",-34.77207,-58.166024
2,"Autopista 25 de Mayo, Buenos Aires Argentina",-34.622955,-58.374269
3,"Avenida Lugones Leopoldo 6900, Buenos Aires, A...",-34.558958,-58.42098
4,"Autopista Perito Moreno, Buenos Aires Argentina",-34.647139,-58.481927


In [177]:
# Los índices específicos en df_hechos que queremos actualizar
indices = [38, 106, 139, 176, 180, 181, 256, 313, 546, 559, 621]

# Mostrar el DataFrame antes de la actualización para comparación
print("Antes de la actualización:")
print(df_hechos.loc[indices, ['pos x', 'pos y']])

# Recorrer los índices y actualizar los valores específicos
for i, idx in enumerate(indices):
    df_hechos.at[idx, 'pos y'] = df.at[i, 'Latitud']
    df_hechos.at[idx, 'pos x'] = df.at[i, 'Longitud']

# Mostrar los valores actualizados en df_hechos
print("\nDespués de la actualización:")
print(df_hechos.loc[indices, ['pos x', 'pos y']])

Antes de la actualización:
    pos x pos y
38    NaN   NaN
106   NaN   NaN
139   NaN   NaN
176   NaN   NaN
180   NaN   NaN
181   NaN   NaN
256   NaN   NaN
313   NaN   NaN
546   NaN   NaN
559   NaN   NaN
621   NaN   NaN

Después de la actualización:
         pos x      pos y
38   -58.42098 -34.558958
106 -58.166024  -34.77207
139 -58.374269 -34.622955
176  -58.42098 -34.558958
180 -58.481927 -34.647139
181 -58.475612  -34.66794
256 -58.378036 -34.634004
313  -58.42098 -34.558958
546  -58.42098 -34.558958
559  -58.71245 -34.508453
621 -58.166024  -34.77207


In [178]:
df_hechos.iloc[[119], :]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20.0,NaN,calle,...,NaN,NaN,NaN,0,NaN,NaN,NaN,peaton-sd,peaton,NaN


In [179]:
df_hechos.drop(index=[119], inplace=True)

In [180]:
df_hechos[df_hechos['COMUNA'].isin([0])]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
139,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0.0,autopista 25 de mayo,autopista,...,NaN,NaN,autopista 25 de mayo,0,NaN,-58.374269,-34.622955,NaN,NaN,NaN


In [181]:
# Agregamos el valor de la comuna
df_hechos.loc[139, 'COMUNA'] = 3

In [182]:
#Verificamos el cambio 
df_hechos.loc[139]

ID                                  2016-0174
N_VICTIMAS                                  1
FECHA                     2016-12-27 00:00:00
AAAA                                     2016
MM                                         12
DD                                         27
HORA                                 00:00:00
HH                                        0.0
LUGAR_DEL_HECHO          autopista 25 de mayo
TIPO_DE_CALLE                       autopista
Calle                    autopista 25 de mayo
Altura                                    NaN
Cruce                                     NaN
Dirección Normalizada    autopista 25 de mayo
COMUNA                                      3
XY (CABA)                                 NaN
pos x                              -58.374269
pos y                              -34.622955
PARTICIPANTES                             NaN
VICTIMA                                   NaN
ACUSADO                                   NaN
Name: 139, dtype: object

In [183]:
sorted(df_hechos['COMUNA'].unique().tolist())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [184]:
df_hechos = df_hechos.drop(columns=['Calle', 'Altura', 'Cruce', 'Dirección Normalizada', 'XY (CABA)', 'LUGAR_DEL_HECHO'])

In [185]:
df_hechos.isna().sum()

ID                0
N_VICTIMAS        0
FECHA             0
AAAA              0
MM                0
DD                0
HORA              1
HH                1
TIPO_DE_CALLE     0
COMUNA            0
pos x             0
pos y             0
PARTICIPANTES     5
VICTIMA           9
ACUSADO          22
dtype: int64

In [186]:
df_hechos[df_hechos['ACUSADO'].isna()]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0.0,avenida,8,-58.46503904,-34.68092974,moto-sd,moto,NaN
32,2016-0045,1,2016-04-11,2016,4,11,10:25:00,10.0,gral paz,13,-58.46743471,-34.53476874,moto-sd,moto,NaN
35,2016-0049,1,2016-04-17,2016,4,17,00:00:00,0.0,autopista,4,-58.37714647568196,-34.63657525428238,NaN,NaN,NaN
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20.0,autopista,13,-58.42098,-34.558958,moto-sd,moto,NaN
57,2016-0080,1,2016-06-18,2016,6,18,06:42:00,6.0,gral paz,9,-58.52927982,-34.63931752,auto-sd,auto,NaN
76,2016-0101,1,2016-08-07,2016,8,7,19:42:00,19.0,gral paz,9,-58.52932872,-34.65014993,NaN,NaN,NaN
83,2016-0110,1,2016-08-24,2016,8,24,20:21:00,20.0,avenida,9,-58.51232237,-34.65903530,peaton-sd,peaton,NaN
92,2016-0119,1,2016-09-04,2016,9,4,06:40:00,6.0,avenida,12,-58.49659234,-34.56353635,NaN,NaN,NaN
105,2016-0135,1,2016-10-24,2016,10,24,05:00:00,5.0,avenida,14,-58.42260225,-34.56403694,peaton-sd,peaton,NaN
112,2016-0143,1,2016-11-06,2016,11,6,04:41:00,4.0,avenida,8,-58.44451316,-34.68475866,moto-sd,moto,NaN


In [187]:
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 695 entries, 0 to 695
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             695 non-null    object        
 1   N_VICTIMAS     695 non-null    int64         
 2   FECHA          695 non-null    datetime64[ns]
 3   AAAA           695 non-null    int64         
 4   MM             695 non-null    int64         
 5   DD             695 non-null    int64         
 6   HORA           694 non-null    object        
 7   HH             694 non-null    float64       
 8   TIPO_DE_CALLE  695 non-null    object        
 9   COMUNA         695 non-null    int64         
 10  pos x          695 non-null    object        
 11  pos y          695 non-null    object        
 12  PARTICIPANTES  690 non-null    object        
 13  VICTIMA        686 non-null    object        
 14  ACUSADO        673 non-null    object        
dtypes: datetime64[ns](1), float6

In [188]:
print(df_hechos['FECHA'].min())
print(df_hechos['FECHA'].max())

2016-01-01 00:00:00
2021-12-30 00:00:00


In [189]:
print(df_hechos['AAAA'].unique())
print(df_hechos['MM'].unique())
valores=sorted(df_hechos['DD'].unique().tolist())
print(valores)
valores2=sorted(df_hechos['HH'].unique().tolist())
print(valores2)

[2016 2017 2018 2019 2020 2021]
[ 1  2  3  4  5  6  7  8  9 10 11 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, nan]


In [190]:
print(f'Hora máxima: {pd.to_datetime(df_hechos["HORA"], format="%H:%M:%S").dt.hour.max()}')
print(f'Hora mínima: {pd.to_datetime(df_hechos["HORA"], format="%H:%M:%S").dt.hour.min()}')
print(f'Minuto máximo: {pd.to_datetime(df_hechos["HORA"], format="%H:%M:%S").dt.minute.max()}')
print(f'Minuto mínimo: {pd.to_datetime(df_hechos["HORA"], format="%H:%M:%S").dt.minute.min()}')
print(f'Segundo máximo: {pd.to_datetime(df_hechos["HORA"], format="%H:%M:%S").dt.second.max()}')
print(f'Segundo mínimo: {pd.to_datetime(df_hechos["HORA"], format="%H:%M:%S").dt.second.min()}')

Hora máxima: 23.0
Hora mínima: 0.0
Minuto máximo: 59.0
Minuto mínimo: 0.0
Segundo máximo: 47.0
Segundo mínimo: 0.0


In [194]:
df_hechos['AAAA'].isna().sum()

0

In [191]:
df_hechos.reset_index(inplace=True, drop=True)

In [192]:
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 695 entries, 0 to 694
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             695 non-null    object        
 1   N_VICTIMAS     695 non-null    int64         
 2   FECHA          695 non-null    datetime64[ns]
 3   AAAA           695 non-null    int64         
 4   MM             695 non-null    int64         
 5   DD             695 non-null    int64         
 6   HORA           694 non-null    object        
 7   HH             694 non-null    float64       
 8   TIPO_DE_CALLE  695 non-null    object        
 9   COMUNA         695 non-null    int64         
 10  pos x          695 non-null    object        
 11  pos y          695 non-null    object        
 12  PARTICIPANTES  690 non-null    object        
 13  VICTIMA        686 non-null    object        
 14  ACUSADO        673 non-null    object        
dtypes: datetime64[ns](1), f

In [193]:
df_hechos.to_csv(r'CleanData/Hechos.csv')

PermissionError: [Errno 13] Permission denied: 'CleanData/Hechos.csv'

In [ ]:
df_victimas=pd.read_excel(r'Data/homicidios.xlsx', sheet_name='VICTIMAS')

In [ ]:
df_victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [ ]:
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [ ]:
print("Valores nulos:\n", df_victimas.isna().sum())
print("\nCantidad de filas duplicadas:\n", df_victimas.duplicated().sum())

Valores nulos:
 ID_hecho               0
FECHA                  0
AAAA                   0
MM                     0
DD                     0
ROL                    0
VICTIMA                0
SEXO                   0
EDAD                   0
FECHA_FALLECIMIENTO    0
dtype: int64

Cantidad de filas duplicadas:
 0


In [ ]:
df_victimas = df_victimas.map(lambda x: x.lower() if isinstance(x, str) else x)

In [ ]:
faltantes = ['', ' ', '.', '...', 'sd', 'sd-sd', 'point (. .)']
df_victimas.replace(faltantes, np.nan, inplace=True)
df_victimas.infer_objects(copy=False)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,conductor,moto,masculino,19.0,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,conductor,auto,masculino,70.0,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,conductor,moto,masculino,30.0,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,conductor,moto,masculino,18.0,NaN
4,2016-0005,2016-01-21,2016,1,21,conductor,moto,masculino,29.0,2016-02-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...
712,2021-0092,2021-12-12,2021,12,12,peaton,peaton,femenino,50.0,2021-12-12 00:00:00
713,2021-0093,2021-12-13,2021,12,13,pasajero_acompañante,moto,femenino,18.0,2021-12-18 00:00:00
714,2021-0094,2021-12-20,2021,12,20,pasajero_acompañante,moto,femenino,43.0,2021-12-20 00:00:00
715,2021-0095,2021-12-30,2021,12,30,conductor,moto,masculino,27.0,2022-01-02 00:00:00


In [ ]:
print("Valores nulos:\n", df_victimas.isna().sum())
print("\nCantidad de filas duplicadas:\n", df_victimas.duplicated().sum())

Valores nulos:
 ID_hecho                0
FECHA                   0
AAAA                    0
MM                      0
DD                      0
ROL                    11
VICTIMA                 9
SEXO                    6
EDAD                   53
FECHA_FALLECIMIENTO    68
dtype: int64

Cantidad de filas duplicadas:
 0


In [ ]:
df_victimas.drop(columns='FECHA_FALLECIMIENTO', inplace=True)

In [ ]:
df_victimas['FECHA'] = pd.to_datetime(df_victimas['FECHA'], errors='coerce')


In [ ]:
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   ID_hecho  717 non-null    object        
 1   FECHA     717 non-null    datetime64[ns]
 2   AAAA      717 non-null    int64         
 3   MM        717 non-null    int64         
 4   DD        717 non-null    int64         
 5   ROL       706 non-null    object        
 6   VICTIMA   708 non-null    object        
 7   SEXO      711 non-null    object        
 8   EDAD      664 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 50.5+ KB


In [ ]:
df_victimas.to_csv(r'CleanData/Victimas.csv')